> Copyright 2022 University of Luxembourg
> 
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
>
>    https://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  
>
***

Author: André Stemper (andre.stemper@uni.lu)

***

# Communication Protocol


**Imports**

In [ ]:
import threading
import queue
import time
import serial
import struct
import numpy as np
import re
from enum import Enum
import signal

**Protocol definitions**

In [ ]:
ENDIANESS = "little"

**Status definitions**

In [ ]:
class PROTCOL_STATUS(object):
    ACK = b'\x30'
    NAK = b'\x31'
    RPLY = b'\x32'
    CNS = b'\x33'

In [ ]:
class IPR(Enum):
    ERROR = 1   # operation failed
    MORE = 2    # more information required to continue
    DONE = 3    # operation done and succeeded
    ALREADY_ACKNOWLEDGED = 4 # done and reception already acknowledged


**Command IDs**

In [ ]:
PROTOCOL_COMMAND_ID_GENERIC = 0x00  # placeholder for a real command
# pseudo command without any data to declare the end of a reply
PROTOCOL_COMMAND_ID_ERP = 0x01
PROTOCOL_COMMAND_ID_INITIALIZE = 0x10  # initialize target  
PROTOCOL_COMMAND_ID_COMMENT = 0x40  # sz string
PROTOCOL_COMMAND_ID_REQUEST_COMMENT = 0x41  # request for comment
PROTOCOL_COMMAND_ID_DATAPOINT_FLOAT = 0x21  # Datapoint object
PROTOCOL_COMMAND_ID_DATAPOINT_INT8 = 0x22   # Datapoint object
# inverse of mahalanobis covariance matrix
PROTOCOL_COMMAND_ID_MAHALANOBIS_INVERSE_COVARIANCE_MATRIX = 0x42
PROTOCOL_COMMAND_ID_MAHALANOBIS_MEAN = 0x43  # mahalanobis mean vector
PROTOCOL_COMMAND_ID_MAHALANOBIS_DISTANCE = 0x83  # mahalanobis distance
PROTOCOL_COMMAND_ID_THRESHOLD = 0x44  # threshold value
PROTOCOL_COMMAND_ID_THRESHOLD_HOLD_OFF = 0x45 # threshold hold-off value
PROTOCOL_COMMAND_ID_3x3_MATRIX = 0x60   # test
PROTOCOL_COMMAND_ID_INT8 = 0x61  # test
PROTOCOL_COMMAND_ID_UINT8 = 0x62  # test
PROTOCOL_COMMAND_ID_FLOAT = 0x63  # test
PROTOCOL_COMMAND_ID_INT32 = 0x64  # test
PROTOCOL_COMMAND_ID_UINT32 = 0x65 # test
PROTOCOL_COMMAND_ID_EXECUTION_TIME = 0x80
# boolean detection result encoded as byte 0:true, 1:false
PROTOCOL_COMMAND_ID_DETECTION = 0x82
PROTOCOL_COMMAND_ID_EXCEPTION = 0xFE  # remote exception 
PROTOCOL_COMMAND_ID_EOC = 0xFF  # end of communication


In [ ]:
def comment(what):
    pass
    # print(what)

**Commands**


In [ ]:
class Command(object):
    def __init__(self, id=PROTOCOL_COMMAND_ID_GENERIC, data=np.zeros(1), expected_data_length=1, callback=None, type_size=1):
        self._id = id
        self._expected_data_length = expected_data_length
        self._callback = callback
        self._type_size = type_size
        self._outgoing_data = data
        self.reset()

    def id(self):
        return self._id

    def encode_header(self):
        return struct.pack('B', self.id())

    def reset(self):
        self._incoming_data = bytearray()

    def begin_reception(self):
        self.reset()
        if self._expected_data_length == 0:
            if not self._callback is None:
                self._callback(None)
            return IPR.DONE
        return IPR.MORE

    def encode(self):
        """ return a byte array to be send """
        pass

    def _decode(self, data):
        return "must be overwritten by subclass"

    def incoming(self, byte):
        self._incoming_data.extend(byte)
        if len(self._incoming_data) == self._expected_data_length:
            value = self._decode()
            if not self._callback is None:
                self._callback(value)
            return IPR.DONE
        return IPR.MORE

    def __repr__(self):
        r = []
        r.append("cmd={}".format(self.id()))
        r.append("in={}".format(self._incoming_data))
        r.append("out={}".format(self._outgoing_data))
        return ','.join(r)


class CommandMatrix(Command):
    """ Command to send/receive matrix / vector """

    def __init__(self, id=PROTOCOL_COMMAND_ID_3x3_MATRIX, data=np.zeros(3*3, dtype=np.float32), callback=None, rows=3, columns=3, type_size=4):
        super().__init__(id, data, expected_data_length=int(
            rows*columns*type_size), callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        for value in self._outgoing_data:
            stream.extend(struct.pack('<f', value))
        return stream

    def _decode(self):
        value = np.zeros(int(self._expected_data_length /
                         self._type_size), dtype=np.float32)
        data = bytes(self._incoming_data)
        for i in range(int(self._expected_data_length/self._type_size)):
            value[i] = struct.unpack_from(
                '<f', data, offset=self._type_size*i)[0]
        return value


class CommandDatapointFloat(Command):
    """ Command to send/receive datapoint """

    def __init__(self, id=PROTOCOL_COMMAND_ID_DATAPOINT_FLOAT, data=np.zeros(9), callback=None, length=9, type_size=4):
        super().__init__(id, data, expected_data_length=int(
            length*type_size), callback=callback)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        for value in self._outgoing_data:
            stream.extend(struct.pack('<f', value))
        return stream

    def _decode(self):
        value = np.zeros(int(self._expected_data_length /
                         self._type_size), dtype=np.float16)
        data = bytes(self._incoming_data)
        for i in range(int(self._expected_data_length/self._type_size)):
            value[i] = struct.unpack_from(
                '<f', data, offset=self._type_size*i)[0]
        return value


class CommandDatapointINT8(Command):
    """ Command to send/receive datapoint """

    def __init__(self,
                 id=PROTOCOL_COMMAND_ID_DATAPOINT_INT8,
                 data=np.zeros(9, dtype=np.int8),
                 callback=None,
                 length=9,
                 type_size=1):
        super().__init__(id, data, expected_data_length=int(
            length*type_size), callback=callback)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        for value in self._outgoing_data:
            stream.extend(struct.pack('B', value))
        return stream

    def _decode(self):
        value = np.zeros(int(self._expected_data_length /
                         self._type_size), dtype=np.float16)
        data = bytes(self._incoming_data)
        for i in range(int(self._expected_data_length/self._type_size)):
            value[i] = struct.unpack_from(
                'B', data, offset=self._type_size*i)[0]
        return value


class CommandInt32(Command):
    """ Command to send/receive int32 """

    def __init__(self, id=PROTOCOL_COMMAND_ID_INT32, data=1, callback=None, type_size=4):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        stream.extend(struct.pack('<i', self._outgoing_data))
        return stream

    def _decode(self):
        return struct.unpack_from('<i', bytes(self._incoming_data), offset=0)[0]


class CommandUInt32(Command):
    """ Command to send/receive uint32 """

    def __init__(self, id=PROTOCOL_COMMAND_ID_UINT32, data=1, callback=None, type_size=4):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        stream.extend(struct.pack('<I', self._outgoing_data))
        return stream

    def _decode(self):
        return struct.unpack_from('<I', bytes(self._incoming_data), offset=0)[0]


class CommandInt8(Command):
    """ Command to send/receive int8 """

    def __init__(self, id=PROTOCOL_COMMAND_ID_INT8, data=1, callback=None, type_size=1):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        stream.extend(struct.pack('b', self._outgoing_data))
        return stream

    def _decode(self):
        return struct.unpack_from('b', bytes(self._incoming_data), offset=0)[0]


class CommandUInt8(Command):
    """ Command to send/receive uint8 """

    def __init__(self, id=PROTOCOL_COMMAND_ID_UINT8, data=1, callback=None, type_size=1):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        stream.extend(struct.pack('B', self._outgoing_data))
        return stream

    def _decode(self):
        return struct.unpack_from('B', bytes(self._incoming_data), offset=0)[0]


class CommandFloat(Command):
    """ Command to send/receive float """

    def __init__(self, id=PROTOCOL_COMMAND_ID_FLOAT, data=0.0, callback=None, type_size=4):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        stream.extend(struct.pack('<f', self._outgoing_data))
        return stream

    def _decode(self):
        return struct.unpack_from('<f', bytes(self._incoming_data), offset=0)[0]


class CommandRequestComment(Command):
    """ Command to request a comment """

    def __init__(self, id=PROTOCOL_COMMAND_ID_REQUEST_COMMENT, data=None, callback=None, type_size=0):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        return stream

    def _decode(self):
        return None


class CommandInitialize(Command):
    """ Command to initialize the target """

    def __init__(self, id=PROTOCOL_COMMAND_ID_INITIALIZE, data=None, callback=None, type_size=0):
        super().__init__(id, data, expected_data_length=type_size,
                         callback=callback, type_size=type_size)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        return stream

    def _decode(self):
        return None


class CommandComment(Command):
    """ Command to send/receive a comment (zero terminated string) """

    def __init__(self, id=PROTOCOL_COMMAND_ID_COMMENT, data="", callback=None):
        super().__init__(id, data, expected_data_length=1, callback=callback, type_size=1)

    def encode(self):
        stream = bytearray()
        stream.extend(self.encode_header())
        for value in self._outgoing_data:
            stream.extend(struct.pack('<B', value))
        stream.append(b'\x00')  # terminate
        return stream

    def incoming(self, byte):
        if byte[0] == 0:
            try:
                string = self._incoming_data.decode('utf8')
            except Exception as e:
                string = "invalid string: {}".format(e)
            if not self._callback is None:
                self._callback(string)
            return IPR.DONE
        self._incoming_data.extend(byte)
        return IPR.MORE


**Connection protocol**


In [ ]:
class ConnectionHandler(object):
    def __init__(self,
                 response_handlers=[],  # dictionary of command_id:command_handler
                 timeout=2.0,
                 timeout_my_turn=0.5,
                 callback_after_emission=None,  # callback after an emission
                 callback_after_reception=None,  # callback after a reception
                 callback_after_emission_of_acknowledge=None):  # callback after emission of an acknowlede

        self.__response_handlers = response_handlers
        self.__timeout = timeout
        self.__timeout_my_turn = timeout_my_turn
        self.__callback_after_emission = callback_after_emission
        self.__callback_after_reception = callback_after_reception
        self.__callback_after_emission_of_acknowledge = callback_after_emission_of_acknowledge
        # holds the currently activ response handler
        self.__current_response_handler = None

        self.__my_turn = threading.Event()
        self.__my_turn.set()  # host side starts the communication

        self.__emit_queue = queue.Queue()
        self.__not_awaiting_status = threading.Event()
        self.__not_awaiting_status.set()
        self.__protocol_status_ack = threading.Event()
        self.__protocol_status_nak = threading.Event()
        self.__protocol_status_rply = threading.Event()
        self.__protocol_status_cns = threading.Event()
        self.__stopped = threading.Event()  # stop the receiving thread
        self.__stopped.clear()

    def open(self):
        """ create and start the main thread """
        self.__thread_emitter = threading.Thread(target=self.__main_emitter, args=())
        self.__thread_emitter.start()

    def close(self):
        """ stop and join the receiving thread """
        self.__close()
        self.__thread_emitter.join()

    def __close(self):
        """ stop and join the receiving thread """
        self.__stopped.set()
        while not self.__emit_queue.empty():
            self.__emit_queue.get()
            self.__emit_queue.task_done()

    def handle_incoming_byte(self, byte):
        """ handle an incoming byte from the microcontroller """
        if self.__my_turn.is_set():
            if not self.__not_awaiting_status.is_set():
                """ awaiting device status """
                if byte == PROTCOL_STATUS.ACK:
                    comment("[>] ACK")
                    self.__protocol_status_ack.set()
                elif byte == PROTCOL_STATUS.NAK:
                    comment("[>] NAK")
                    self.__protocol_status_nak.set()
                elif byte == PROTCOL_STATUS.RPLY:
                    comment("[>] RPLY")
                    self.__protocol_status_rply.set()
                    self.__my_turn.clear()
                elif byte == PROTCOL_STATUS.CNS:
                    comment("[>] CNS")
                    self.__protocol_status_cns.set()
                else:
                    raise Exception("Invalid status response from device... {}".format(
                        hex(int.from_bytes(byte, ENDIANESS, signed=False))))
                self.__not_awaiting_status.set()
                return
            else:
                raise Exception("MC disrespects the protocol.")
        elif self.__current_response_handler is None:
            """ no handler selected: awaiting command ID """
            if int.from_bytes(byte, ENDIANESS, signed=False) == PROTOCOL_COMMAND_ID_ERP:
                """ this is the end of a reply """
                comment("[>] ERP")
                self.__my_turn.set()  
                self.emit_status(PROTCOL_STATUS.ACK)
                return
            else:
                """ select handler for this command ID """
                cmd = int.from_bytes(byte, ENDIANESS, signed=False)
                comment("[>] CMD ID {}".format(hex(cmd)))
                for handler in self.__response_handlers:
                    if handler.id() == cmd:
                        self.__current_response_handler = handler
                if self.__current_response_handler is None:
                    raise Exception(
                        "No command handler registered for received command {}.".format(hex(cmd)))
                # begin reception (remove the data from last transaction)
                status = self.__current_response_handler.begin_reception()
                if status == IPR.ALREADY_ACKNOWLEDGED:
                    # command without argument already acknowledged
                    self.__current_response_handler = None
                    return
                if status == IPR.DONE:
                    # command without argument not acknowledged
                    self.__current_response_handler = None
                    self.emit_status(PROTCOL_STATUS.ACK)
                    return
                # more data required
                self.emit_status(PROTCOL_STATUS.ACK)
                return
        else:
            """ a handler has been selected: all following data is redirected to this handler """
            # comment(" {} => forwarded to handler".format(hex(int.from_bytes(byte, ENDIANESS, signed=False))))
            status = self.__current_response_handler.incoming(byte)
            if status != IPR.MORE:
                self.__current_response_handler = None

        if not self.__callback_after_reception is None:
            self.__callback_after_reception(byte)
        self.emit_status(PROTCOL_STATUS.ACK)

    def emit_status(self, status):
        self._emit_byte(status)
        if not self.__callback_after_emission_of_acknowledge is None:
            self.__callback_after_emission_of_acknowledge(status)

    def emit(self, command, blocking=True):
        """ emit a command or a list of commands """
        if isinstance(command, Command):
            self.__emit_queue.put(command)
        elif isinstance(command, list):
            for cmd in command:
                self.__emit_queue.put(cmd)
        else:
            raise ValueError
        if blocking:
            # self.__emit_queue.join() # this is blocking even if queue emptied during exception
            while not self.__stopped.is_set():
                if self.__emit_queue.empty():
                    break
                time.sleep(0.1)
        return True

    def __main_emitter(self):
        """ main loop to empit commands from the command queue """
        try:
            while not self.__stopped.is_set():  # while this thread is not stopped
                if not self.__emit_queue.empty():  # and there are objects to emit
                    # wait for my turn
                    if self.__my_turn.wait(timeout=self.__timeout_my_turn):
                        command = self.__emit_queue.get()  # then get the first command
                        self.__emit(command)  # and emit it
                        self.__emit_queue.task_done()
                else:
                    time.sleep(0.005)  # and then have a small break (communication is not fast anyway)
        except Exception as e:
            self.__close()
            raise(e)

    def join(self):
        self.__emit_queue.join()

    def __emit(self, command):
        """ emit a command """
        # send command
        for c in bytes(command.encode()):  # for all bytes in the encoded object
            c = int(c).to_bytes(1, byteorder=ENDIANESS, signed=False)
            # clear flags
            self.__protocol_status_ack.clear()
            self.__protocol_status_nak.clear()
            self.__protocol_status_rply.clear()
            self.__protocol_status_cns.clear()
            self.__not_awaiting_status.clear()  # status from device pending

            # emit one byte
            self._emit_byte(c)

            # callback after emission
            if not self.__callback_after_emission is None:
                self.__callback_after_emission(self, c)

            # wait for ACK | NAK | RPLY | <timeout>
            if not self.__not_awaiting_status.wait(timeout=self.__timeout):
                raise Exception("Timeout waiting for status ...")
            if self.__protocol_status_cns.is_set():
                raise Exception("MC does not support the command {}".format(
                    hex(int.from_bytes(c, ENDIANESS, signed=False))))
            if self.__protocol_status_rply.is_set():
                break

        return True

    def _emit_byte(self, byte):
        """ emit one byte: to be overwritten by derived class """
        comment("emit({}).".format(
            hex(int.from_bytes(byte, ENDIANESS, signed=False))))


## Data link layers


**Communication over serial port**


In [ ]:
class ConnectionSerial(ConnectionHandler):
    def __init__(self, response_handlers={}, port="/dev/ttyUSB0", baudrate=115200, inter_byte_timeout=2):
        super().__init__(response_handlers=response_handlers)
        self.__port = port
        self.__baudrate = baudrate
        self.__stopped = threading.Event()
        self.__thread = threading.Thread(target=self.__input_handler, args=())
        self.__connection = serial.Serial(self.__port, baudrate,
                                          inter_byte_timeout=inter_byte_timeout)
        self.__thread.start()

    def close(self):
        if not self.__stopped.is_set():
            try:
                self.__stopped.set()
                self.__thread.join()
            except:
                pass
        try:
            self.__connection.close()
        except:
            pass

    def __close(self):
        self.__stopped.set()
        try:
            self.__connection.close()
        except:
            pass
        super().close()

    def close(self):
        self.__close()
        try:
            self.__thread.join()
        except:
            pass
        super().close()

    def __del__(self):
        self.close()

    def _emit_byte(self, byte):
        self.__connection.write(byte)
        self.__connection.flush()
        comment("[<] {}".format(hex(int.from_bytes(byte, ENDIANESS, signed=False))))

    def __input_handler(self):
        while not self.__stopped.is_set():
            try:
                if self.__connection.in_waiting:
                    byte = self.__connection.read(1)
                    comment("[>] {}".format(hex(int.from_bytes(byte, ENDIANESS, signed=False))))
                    self.handle_incoming_byte(byte)
                else:
                    time.sleep(0.0001)
            except Exception as e:
                self.__stopped.set()
                raise(e)



**Communication over pipe**


In [ ]:
import subprocess
import select
import fcntl
import os
import sys


class ConnectionSubprocess(ConnectionHandler):
    def __init__(self, command=["./protocol_test"], response_handlers={}):
        super().__init__(response_handlers=response_handlers)
        self.__stopped = threading.Event()
        self.__process = subprocess.Popen(command,
                                          shell=False,
                                          stdin=subprocess.PIPE,
                                          stdout=subprocess.PIPE,
                                          stderr=sys.stdout,
                                          universal_newlines=False)
        # give the process some time to arrive at the main loop
        time.sleep(1.0)
        if not self.__process.poll() is None:
            raise Exception("Failed to start supprocess {}".format(command))
        # make the process' stdout non blocking
        fd = self.__process.stdout.fileno()
        fl = fcntl.fcntl(fd, fcntl.F_GETFL)
        fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)
        fd = self.__process.stdin.fileno()
        fl = fcntl.fcntl(fd, fcntl.F_GETFL)
        fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)
        # receiving thread
        self.__thread = threading.Thread(target=self.__input_handler, args=())
        self.__thread.start()

    def __close(self):
        self.__stopped.set()
        super().close()

    def close(self):
        self.__close()
        try:
            self.__thread.join()
        except:
            pass
        super().close()

        comment("Stopping subprocess {}".format(self.__process.pid))
        try:
            if self.__process.poll() is None:
                self.__process.send_signal(signal.SIGINT)
                time.sleep(0.5)
        except Exception as e:
            print(e)
            pass
        try:
            if self.__process.poll() is None:
                os.killpg(os.getpgid(self.__process.pid), signal.SIGINT)
                time.sleep(0.5)
        except:
            pass
        try:
            if self.__process.poll() is None:
                os.killpg(os.getpgid(self.__process.pid), signal.SIGTERM)
        except:
            pass

    def __del__(self):
        self.close()

    def _emit_byte(self, byte):
        self.__process.stdin.write(byte)
        self.__process.stdin.flush()
        comment("[<] {}".format(hex(int.from_bytes(byte, ENDIANESS, signed=False))))

    def __input_handler(self, time_limit=0.5):
        while not self.__stopped.is_set():
            # check if subprocess has terminated
            if not self.__process.poll() is None:
                self.__close()
                break
            # wait and process incoming data
            try:
                sel_readers = [self.__process.stdout]
                sel_writers = []
                sel_error = [] 
                ready_readers, ready_writters, ready_errors = select.select(
                    sel_readers, sel_writers, sel_error, time_limit)
                if len(ready_readers) > 0:
                    # a single select can announce multiple bytes. 
                    # they have to be read here as this will not trigger another select .
                    while (not self.__process.stdout.peek(1) is None) and (not self.__process.stdout.peek(1) == b''):
                        byte = self.__process.stdout.read(1)
                        comment("[>] {}".format(
                            hex(int.from_bytes(byte, ENDIANESS, signed=False))))
                        self.handle_incoming_byte(byte)
            except Exception as e:
                self.__stopped.set()
                raise(e)


# Examples


In [ ]:
if not 'enable_example' in locals():
    enable_example = True


**Example communication over serial port**


In [ ]:
enable_example_serial = False


In [ ]:
if enable_example and enable_example_serial:
    response_handlers = [
        CommandInt32(PROTOCOL_COMMAND_ID_INT32,
                     callback=lambda d: print("Got int32={}".format(d))),
        CommandFloat(PROTOCOL_COMMAND_ID_FLOAT,
                     callback=lambda d: print("Got float={}".format(d)))
    ]

    connection = ConnectionSerial(port="/dev/ttyUSB0",
                                  baudrate=115200,
                                  response_handlers=response_handlers)
    try:
        connection.open()

        commands = [
            CommandDatapointINT8(data=np.array(
                [40, 40, 41, 42, 41, 41, 40, 40, 39])),
            CommandDatapointINT8(data=np.array(
                [40, 40, 41, 42, 41, 41, 40, 40, 39])),
            CommandFloat(data=42.21)
        ]
        connection.emit(commands)

        time.sleep(2)  # sleep to see dome output on jupyter n.b.
        input("Press enter to continue")
    finally:
        connection.close()
    print("done")


**Example communication over pipe**  
This requires the C++ implementation of "protocol_test" to be compiled for the host machine.


In [ ]:
enable_example_pipe = True


In [ ]:

if enable_example and enable_example_pipe:
    response_handlers = [
        CommandInt32(PROTOCOL_COMMAND_ID_INT32,
                     callback=lambda d: print("Got int32={}".format(d))),
        CommandFloat(PROTOCOL_COMMAND_ID_FLOAT,
                     callback=lambda d: print("Got float={}".format(d)))
    ]

    connection = ConnectionSubprocess(command=["./protocol_test"],
                                      response_handlers=response_handlers)
    try:
        connection.open()

        commands = [
            CommandDatapointINT8(data=np.array(
                [40, 40, 41, 42, 41, 41, 40, 40, 39])),
            CommandDatapointINT8(data=np.array(
                [40, 40, 41, 42, 41, 41, 40, 40, 39])),
            CommandFloat(data=42.21)
        ]
        connection.emit(commands)
        connection.join()
    finally:
        connection.close()
